# XPECTO Epidemic 2.0: Practice Strategies

This notebook provides a specialized environment for testing your epidemic intervention strategies. All attempts in this notebook are in practice mode and will not count toward your official competition score.

## Setup

First, let's set up our environment and import the necessary modules.

In [25]:
# Add the project root to the path
import sys
import os
from pathlib import Path

# Add parent directory to path for imports
sys.path.append(str(Path.cwd().parent))

# Import required modules
from src.competition import CompetitionManager
from src.competition.testing.enhanced_engine import EnhancedEngine as Engine

# Import visualization libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import src.competition.utils as utils

# Set up visualization
plt.style.use('seaborn-whitegrid')
sns.set(font_scale=1.2)
pd.set_option('display.max_columns', None)

## Initialize Competition System

Now, let's initialize the competition system and register as a player.

In [26]:
# Create an epidemic engine
engine = Engine()

# Create competition manager with a practice data directory
competition = CompetitionManager(data_dir="practice_data", engine=engine)

# Register as a player - use your real name for consistent tracking
player_name = "Your Name"  # Change this to your name
player_id = competition.setup_player(name=player_name)
print(f"Registered as player: {player_name} (ID: {player_id})")

# Ensure we're in practice mode
competition.toggle_practice_mode(is_practice=True)
print("Practice mode enabled - attempts will not count for competition")

Player registered: Your Name (ID: 155666fe-680c-48ec-8d45-90867a656b89)
Registered as player: Your Name (ID: 155666fe-680c-48ec-8d45-90867a656b89)
Mode set to: Practice
Practice mode enabled - attempts will not count for competition


## Available Scenarios

Let's check what scenarios are available for us to test.

In [27]:
# Get available scenarios
scenarios = competition.list_scenarios()
display(pd.DataFrame(scenarios)[['ID', 'Name', 'Difficulty', 'R0', 'Resources']])

# Display full scenario details
scenarios

,ID,Name,Difficulty,R0,Resources
0,standard,Standard Outbreak,standard,2.5,1000
1,challenging,Challenging Outbreak,challenging,3.5,700


,ID,Name,Difficulty,R0,Resources
0,standard,Standard Outbreak,standard,2.5,1000
1,challenging,Challenging Outbreak,challenging,3.5,700


[{'ID': 'standard',
  'Name': 'Standard Outbreak',
  'Difficulty': 'standard',
  'R0': 2.5,
  'Resources': 1000},
 {'ID': 'challenging',
  'Name': 'Challenging Outbreak',
  'Difficulty': 'challenging',
  'R0': 3.5,
  'Resources': 700}]

## Strategy Testing Framework

This section provides a framework for testing various epidemic intervention strategies. You can define, test, and compare multiple approaches to find the most effective one.

### Basic Strategy: Simple Fixed Approach

Let's start with a basic strategy that applies a fixed set of interventions.

In [28]:
def basic_strategy(engine):
    """A basic intervention strategy that applies a set of fixed measures."""
    
    # Set moderate lockdown
    engine.set_lockdown_level(0.5)
    
    # Allocate resources to healthcare
    engine.allocate_resources('healthcare', 200)
    
    # Allocate some resources to research
    engine.allocate_resources('research', 100)
    
    # Restrict travel
    if hasattr(engine, 'restrict_travel'):
        engine.restrict_travel(True)

In [29]:
# Set up a simulation with the standard scenario
competition.set_scenario("standard")
competition.setup_simulation()

# Run the simulation with our basic strategy
basic_results = competition.run_simulation(steps=365, interventions=[basic_strategy])

# Display the results
competition.display_score(basic_results)

Current scenario set to: Standard Outbreak (standard)
Scenario 'Standard Outbreak' configured with slight variations:
  - R0: 2.50 (base: 2.5)
  - Initial infections: 94 across regions (base: 100)
  - Resources: 988 (base: 1000)
Simulation configured for scenario: Standard Outbreak

=== Active Disease Variants ===
Gamma Variant:
  Prevalence: 100.0%
  Transmissibility: 1.7x base
  Severity: 1.2x base
  Immune Escape: 50.0%

Beta Variant:
  Prevalence: 98.5%
  Transmissibility: 1.3x base
  Severity: 1.3x base
  Immune Escape: 30.0%



### Adaptive Strategy: Responding to Changing Conditions

Now let's create a more dynamic strategy that adapts to the current state of the epidemic.

In [30]:
def adaptive_strategy(engine):
    """A strategy that adapts based on the current infection rate."""
    
    # Initial measures
    engine.set_lockdown_level(0.3)  # Start with lighter restrictions
    engine.allocate_resources('healthcare', 150)
    engine.allocate_resources('research', 50)
    
    # Create a state tracker to manage our strategy across steps
    state_tracker = {
        'peak_infections': 0.0,
        'current_lockdown': 0.3,
        'healthcare_invested': 150,
        'research_invested': 50,
        'economic_invested': 0
    }
    
    # Define step callback function to adjust strategy
    def monitor_and_respond(step, state):
        # Calculate current infection rate
        try:
            total_population = state.population.total
            infected = state.population.infected
            infection_rate = infected / total_population
            
            # Update peak tracking
            if infection_rate > state_tracker['peak_infections']:
                state_tracker['peak_infections'] = infection_rate
            
            # Adjust lockdown based on infection rate
            if infection_rate > 0.15:  # Severe outbreak
                target_lockdown = 0.8
            elif infection_rate > 0.1:  # Serious outbreak
                target_lockdown = 0.7
            elif infection_rate > 0.05:  # Moderate outbreak
                target_lockdown = 0.5
            elif infection_rate > 0.02:  # Mild outbreak
                target_lockdown = 0.3
            else:  # Controlled situation
                target_lockdown = 0.2
            
            # Don't change lockdown too abruptly (max 0.1 change per step)
            current = state_tracker['current_lockdown']
            change = target_lockdown - current
            change = max(min(change, 0.1), -0.1)  # Limit change magnitude
            new_lockdown = current + change
            state_tracker['current_lockdown'] = new_lockdown
            
            # Apply the new lockdown level
            engine.set_lockdown_level(new_lockdown)
            
            # Adjust resource allocation based on stage of epidemic and needs
            if step < 50:  # Early phase
                # Prioritize research and detection
                engine.allocate_resources('research', 10)
                state_tracker['research_invested'] += 10
            elif infection_rate > 0.1:  # High infection phase
                # Prioritize healthcare
                engine.allocate_resources('healthcare', 20)
                state_tracker['healthcare_invested'] += 20
            elif new_lockdown > 0.5:  # Significant lockdown
                # Provide economic support during strict measures
                engine.allocate_resources('economic', 15)
                state_tracker['economic_invested'] += 15
            else:  # Recovery or controlled phase
                # Balanced approach
                engine.allocate_resources('healthcare', 5)
                engine.allocate_resources('economic', 5)
                state_tracker['healthcare_invested'] += 5
                state_tracker['economic_invested'] += 5
            
            # Apply travel restrictions during high infection
            if hasattr(engine, 'restrict_travel'):
                engine.restrict_travel(infection_rate > 0.05)
                
        except Exception as e:
            # Fallback if the state structure is different
            print(f"Error adjusting strategy: {e}")
    
    # Register our callback
    engine.register_step_callback(monitor_and_respond)

In [31]:
# Set up and run the simulation with our adaptive strategy
competition.set_scenario("standard")
competition.setup_simulation()
adaptive_results = competition.run_simulation(steps=365, interventions=[adaptive_strategy])

# Display the results
competition.display_score(adaptive_results)

Current scenario set to: Standard Outbreak (standard)
Scenario 'Standard Outbreak' configured with slight variations:
  - R0: 2.50 (base: 2.5)
  - Initial infections: 94 across regions (base: 100)
  - Resources: 988 (base: 1000)
Simulation configured for scenario: Standard Outbreak
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: div

### Regional Strategy: Location-Based Interventions

This strategy applies different measures to different regions based on their characteristics.

In [32]:
def regional_strategy(engine):
    """A strategy that applies different measures to different regions."""
    
    # Initial global measures
    engine.set_lockdown_level(0.4)  # Moderate global lockdown
    engine.allocate_resources('healthcare', 100)
    engine.allocate_resources('research', 50)
    
    # Try to apply region-specific lockdown if supported
    if hasattr(engine, 'set_regional_lockdown'):
        engine.set_regional_lockdown('capital', 0.7)  # Strict in dense areas
        engine.set_regional_lockdown('major_city_1', 0.6)
        engine.set_regional_lockdown('major_city_2', 0.6)
        engine.set_regional_lockdown('rural_area_1', 0.3)  # Lighter in rural areas
    
    # Try to allocate regional resources if supported
    if hasattr(engine, 'allocate_regional_resources'):
        engine.allocate_regional_resources('capital', 'healthcare', 200)
        engine.allocate_regional_resources('major_city_1', 'healthcare', 150)
        engine.allocate_regional_resources('major_city_2', 'healthcare', 150)
        engine.allocate_regional_resources('rural_area_1', 'healthcare', 50)
    
    # Define a callback to adjust based on regional data
    def regional_monitor(step, state):
        try:
            # Check if we can access regional state data
            if hasattr(engine, 'enhanced_state') and 'regional_infection' in engine.enhanced_state:
                regional_data = engine.enhanced_state['regional_infection']
                
                # Apply targeted measures based on regional infection rates
                for region, infection_rate in regional_data.items():
                    if infection_rate > 0.15:  # Severe regional outbreak
                        if hasattr(engine, 'set_regional_lockdown'):
                            engine.set_regional_lockdown(region, 0.8)
                        if hasattr(engine, 'allocate_regional_resources'):
                            engine.allocate_regional_resources(region, 'healthcare', 30)
                    elif infection_rate < 0.02:  # Well-controlled region
                        if hasattr(engine, 'set_regional_lockdown'):
                            engine.set_regional_lockdown(region, 0.2)
        except Exception as e:
            # Fall back to global approach if regional features aren't available
            pass
    
    # Register the callback
    engine.register_step_callback(regional_monitor)

In [33]:
# Run the regional strategy simulation
competition.set_scenario("standard")
competition.setup_simulation()
regional_results = competition.run_simulation(steps=365, interventions=[regional_strategy])

# Display results
competition.display_score(regional_results)

Current scenario set to: Standard Outbreak (standard)
Scenario 'Standard Outbreak' configured with slight variations:
  - R0: 2.50 (base: 2.5)
  - Initial infections: 94 across regions (base: 100)
  - Resources: 988 (base: 1000)
Simulation configured for scenario: Standard Outbreak
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: div

### Balance Strategy: Economic vs. Health Tradeoffs

This strategy balances health outcomes with economic concerns.

In [34]:
def balance_strategy(engine):
    """A strategy balancing health and economic outcomes."""
    
    # Initial measures - moderate approach
    engine.set_lockdown_level(0.4)
    engine.allocate_resources('healthcare', 100)
    engine.allocate_resources('economic', 100)
    engine.allocate_resources('research', 100)
    
    # Track state between callbacks
    state_data = {
        'economic_health': 1.0,  # Start at 100%
        'healthcare_capacity': 1.0,  # Start at 100%
        'vaccine_progress': 0.0,  # Start at 0%
        'phase': 'early'  # Track epidemic phase
    }
    
    def adaptive_balance(step, state):
        try:
            # Update our state tracking
            if hasattr(state, 'economy') and hasattr(state.economy, 'current_gdp'):
                state_data['economic_health'] = state.economy.current_gdp / state.economy.initial_gdp
            
            if hasattr(state, 'healthcare') and hasattr(state.healthcare, 'capacity'):
                state_data['healthcare_capacity'] = state.healthcare.available_capacity / state.healthcare.total_capacity
            
            if hasattr(state, 'research') and hasattr(state.research, 'vaccine_progress'):
                state_data['vaccine_progress'] = state.research.vaccine_progress
            
            # Calculate infection metrics
            infection_rate = state.population.infected / state.population.total
            
            # Determine phase
            if step < 60:
                state_data['phase'] = 'early'
            elif infection_rate > 0.1:
                state_data['phase'] = 'crisis'
            elif state_data['vaccine_progress'] > 0.7:
                state_data['phase'] = 'late'
            else:
                state_data['phase'] = 'middle'
            
            # Adjust strategy based on current conditions
            if state_data['phase'] == 'early':
                # Early phase: focus on research and preparation
                engine.set_lockdown_level(0.3)
                engine.allocate_resources('research', 20)
                engine.allocate_resources('healthcare', 10)
            
            elif state_data['phase'] == 'crisis':
                # Crisis phase: prioritize healthcare but consider economic balance
                if state_data['economic_health'] < 0.5:
                    # Economy is suffering severely - moderate approach
                    engine.set_lockdown_level(0.5)
                    engine.allocate_resources('economic', 20)
                    engine.allocate_resources('healthcare', 15)
                else:
                    # Economy can withstand stricter measures
                    engine.set_lockdown_level(0.7)
                    engine.allocate_resources('healthcare', 25)
                    engine.allocate_resources('economic', 10)
            
            elif state_data['phase'] == 'middle':
                # Middle phase: balanced approach
                engine.set_lockdown_level(0.4)
                engine.allocate_resources('healthcare', 15)
                engine.allocate_resources('economic', 15)
                engine.allocate_resources('research', 10)
            
            else:  # Late phase
                # Late phase: focus on economic recovery
                engine.set_lockdown_level(0.2)
                engine.allocate_resources('economic', 20)
                engine.allocate_resources('healthcare', 10)
        
        except Exception as e:
            # Fallback if state structure is different
            pass
    
    # Register callback
    engine.register_step_callback(adaptive_balance)

In [35]:
# Run the balanced strategy simulation
competition.set_scenario("standard")
competition.setup_simulation()
balance_results = competition.run_simulation(steps=365, interventions=[balance_strategy])

# Display results
competition.display_score(balance_results)

Current scenario set to: Standard Outbreak (standard)
Scenario 'Standard Outbreak' configured with slight variations:
  - R0: 2.50 (base: 2.5)
  - Initial infections: 94 across regions (base: 100)
  - Resources: 988 (base: 1000)
Simulation configured for scenario: Standard Outbreak
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: division by zero
Error adjusting strategy: div

## Compare All Strategies

Let's compare the results of all strategies we've tested.

In [36]:
# Prepare strategies dictionary for comparison
strategies = {
    "Basic Fixed": basic_strategy,
    "Adaptive": adaptive_strategy,
    "Regional": regional_strategy

SyntaxError: incomplete input (1480223904.py, line 5)